In [7]:
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace

from langgraph.graph import StateGraph, START, END
from typing import TypedDict
import os
from dotenv import load_dotenv

In [8]:
load_dotenv()

llm= HuggingFaceEndpoint(
    repo_id="meta-llama/Llama-3.2-3B-Instruct",
    task="text-generation",
    huggingfacehub_api_token= os.getenv("HUGGINGFACE_API_ENDPOINT"),
    max_new_tokens= 500,
    temperature= 0.3
)

model= ChatHuggingFace(llm= llm)

In [9]:
# create a state
class LLMState(TypedDict):
    question: str
    answer: str

In [10]:
def llm_qa(state:LLMState)-> LLMState:
    # extract the question from the state
    que= state['question']
    
    # give it to the model
    answer= model.invoke(que).content
    
    # save the answer in the state
    state['answer']= answer
    
    return state

In [11]:
# create a graph
graph= StateGraph(LLMState)

# add nodes
graph.add_node('llm_qa', llm_qa)

# add edges
graph.add_edge(START, 'llm_qa')
graph.add_edge('llm_qa',END)

# compile
workflow= graph.compile()



In [12]:
# execute
initial_state={'question': 'Tell me about the meditation'}
final_state= workflow.invoke(initial_state)

final_state

{'question': 'Tell me about the meditation',
 'answer': 'There are many types of meditation practices, and I\'ll provide an overview of some of the most popular ones:\n\n1. **Mindfulness Meditation**: This involves paying attention to the present moment, often by focusing on the breath, body sensations, or emotions. The goal is to cultivate awareness and acceptance of the present moment, without judgment or attachment.\n2. **Loving-Kindness Meditation**: Also known as Metta Meditation, this practice involves cultivating feelings of love, compassion, and kindness towards oneself and others. The goal is to develop a sense of inner peace and connection with others.\n3. **Transcendental Meditation**: This is a technique developed by Maharishi Mahesh Yogi, which involves the use of a mantra to quiet the mind and access a deeper state of consciousness. The goal is to experience a state of inner peace and unity.\n4. **Concentration Meditation**: This involves focusing the mind on a single poi